In [1]:
import pandas as pd
import spacy
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [2]:
prqt = pd.read_parquet('../data/parquet/MultL/test_multilingual.parquet', engine='fastparquet')

In [3]:
prqt_pt = prqt[prqt['lang'] == 'pt']

In [4]:
def map_tag(tag_id):
    if tag_id in range(1, 3):
        return "PER"
    elif tag_id in range(3, 5):
        return "ORG"
    elif tag_id in range(5, 7):
        return "LOC"
    elif tag_id in range(7, 9):
        return "MIST"
    else:
        return "O"

In [5]:
def convert_tokens_to_spacy_format(tokens, tags):
    text = ""
    entities = []
    token_start = 0

    for i, (token, tag_id) in enumerate(zip(tokens, tags)):
        tag = map_tag(tag_id)

        # Calcula os offsets
        start = len(text)
        text += token
        end = len(text)

        if i < len(tokens) - 1:
            text += " "

        if tag != "O":
            # Se for entidade, adiciona aos spans
            entities.append((start, end, tag))

    return (text.strip(), {"entities": entities})

In [6]:
data_spacy = [
    convert_tokens_to_spacy_format(row["tokens"], row["ner_tags"])
    for _, row in prqt_pt.iterrows()
]

for item in data_spacy[:5]:
    print(item)


('Por volta de 1874 , a sede do governo do condado foi transferida de Mineola para Long Island City .', {'entities': [(68, 75, 'LOC'), (81, 85, 'LOC'), (86, 92, 'LOC'), (93, 97, 'LOC')]})
('Na população nascida no exterior , 49,5 % nasceram na América Latina , 33,5 % na Ásia , 14,8 % na Europa , 1,8 % na África e 0,4 % na América do Norte .', {'entities': [(54, 61, 'LOC'), (62, 68, 'LOC'), (81, 85, 'LOC'), (98, 104, 'LOC'), (116, 122, 'LOC'), (134, 141, 'LOC'), (142, 144, 'LOC'), (145, 150, 'LOC')]})
('Além disso , 51,2 % da população nasceu nos Estados Unidos .', {'entities': [(44, 51, 'LOC'), (52, 58, 'LOC')]})
('A partir da sede na Suíça a entidade se tornou uma rede mundial de defesa do meio ambiente , com representações nos principais países do mundo .', {'entities': [(20, 25, 'LOC')]})
('Em 2014 , citando condições desatualizadas nos terminais do aeroporto , o vice-presidente Joe Biden comparou o Aeroporto LaGuardia a " país de terceiro mundo " .', {'entities': [(90, 93, 'PER'), 

In [7]:
TRAIN_DATA = data_spacy[:int(len(data_spacy) * 0.90)]
DEV_DATA = data_spacy[:int(len(data_spacy) * 0.10)]

In [8]:
def create_docbin(data, file_path):
    nlp = spacy.blank("pt")
    db = DocBin()

    print(f"Gerando dados para {file_path}...")
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is not None:
                ents.append(span)
        doc.ents = ents
        db.add(doc)

    db.to_disk(file_path)
    print(f"Dados salvos em '{file_path}'")

In [9]:
create_docbin(TRAIN_DATA, "../train.spacy")
create_docbin(DEV_DATA, "../dev.spacy")

c:\Users\User\Desktop\git\codigos\faculdade\bigdata-tmp\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Gerando dados para ../train.spacy...


100%|██████████| 9144/9144 [00:00<00:00, 10204.99it/s]


Dados salvos em '../train.spacy'
Gerando dados para ../dev.spacy...


100%|██████████| 1016/1016 [00:00<00:00, 7784.65it/s]

Dados salvos em '../dev.spacy'


In [7]:
import spacy
# C:/Users/mathe/OneDrive/Desktop/git/codigos/faculdade/bigdata-tmp/model-1/model-best
nlp = spacy.load("C:/Users/mathe/OneDrive/Desktop/git/codigos/faculdade/bigdata-tmp/model-1/model-best")

doc = nlp("A OpenAI é uma empresa maravilhosa que o João trabalha")

for ent in doc.ents:
    print(ent.text, ent.label_)


OpenAI ORG
João PER


In [ ]:
!python -m spacy train ./config.cfg --output ./model --paths.train ./traning/train_data.spacy --paths.dev ./traning/dev_data.spacy

In [ ]:
!python -m spacy train config_trf.cfg --paths.train train.spacy --paths.dev dev.spacy --gpu-id 0 -o training